In [6]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [7]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS


embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vector_store = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

In [8]:
# In actual usage, you would set `k` to be a higher value, but we use k=1 to show that
# the vector lookup still returns the semantically relevant information
retriever = vector_store.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# When added to an agent, the memory object can save pertinent information from conversations or used tools
memory.save_context({"input": "我最喜欢披萨."}, {"output": "很好啊"})

In [9]:
# Notice the first result returned is the memory pertaining to tax help, which the language model deems more semantically relevant
# to a 1099 than the other documents, despite them both containing numbers.
print(memory.load_memory_variables({"prompt": "我喜欢什么"})["history"])

input: 我最喜欢披萨.
output: 很好啊


In [10]:
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. Reply in Chinese.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm,
    prompt=PROMPT,
    # We set a very low max_token_limit for the purposes of testing.
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="我最喜欢吃什么?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know. Reply in Chinese.

Relevant pieces of previous conversation:
input: 我最喜欢披萨.
output: 很好啊

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 我最喜欢吃什么?
AI:

> Finished chain.


'根据我的记录，您最喜欢的食物是披萨。不过，如果您想尝试一些新的食物，我可以为您推荐一些不同的菜肴，比如中餐、意大利面、墨西哥卷饼等等。您有什么偏好吗？'